## Bottle Cap Color Detection for YOLOv8 (Model Development)

## Project Introduction

This project focuses on developing a robust object detection model for **bottle cap color detection using YOLOv8**. The primary goal is to accurately identify and classify bottle caps based on their colors, such as light blue, dark blue, and other colors.

### Purpose and Importance

Accurate bottle cap color detection is crucial for various industrial applications, including:

*   **Quality Control:** Ensuring that bottles are sealed with the correct colored caps, preventing packaging errors.
*   **Automated Sorting:** Facilitating the automatic sorting of products based on cap color, streamlining manufacturing processes.
*   **Inventory Management:** Aiding in the automated tracking and management of products by their cap specifications.

### Technology

We leverage **YOLOv8 (You Only Look Once version 8)**, a state-of-the-art, real-time object detection algorithm. YOLOv8 is chosen for its superior speed and accuracy, making it an ideal candidate for deployment in environments requiring quick processing, such as on edge devices like the Raspberry Pi 5. The project includes training a custom YOLOv8 model, optimizing it for performance, and evaluating its efficiency in detecting bottle cap colors.

### Environment Setup

To ensure a smooth model development and training process, it's crucial to set up the environment correctly. This involves verifying GPU availability and installing all necessary libraries.

#### 1. Verify GPU Availability

GPU acceleration significantly speeds up model training. Before proceeding, confirm that a GPU is available and properly configured:

- **Check CUDA availability**: Use `torch.cuda.is_available()` to determine if PyTorch can detect a CUDA-compatible GPU.
- **Identify GPU**: If a GPU is available, you can get its name and memory using `torch.cuda.get_device_name(0)` and `torch.cuda.get_device_properties(0).total_memory / 1e9`.
- **Action**: If no GPU is found or it's not detected, consider changing the Colab runtime type to include a GPU (e.g., T4 GPU).

#### 2. Install Required Libraries

Several Python libraries are essential for YOLOv8 model development, training, and deployment. These can be installed using `pip`:

- `ultralytics`: The core library for YOLOv8 models.
- `wandb`: For experiment tracking and visualization (Weights & Biases).
- `onnx`: For Open Neural Network Exchange format handling, crucial for model export.
- `onnxruntime`: The ONNX runtime for efficient model inference.
- `onnxruntime-tools`: Additional tools for ONNX, including quantization.

These steps ensure that your environment is fully prepared for the subsequent stages of model development, training, and optimization.

In [ ]:
import torch

print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✅ GPU READY!")
else:
    print("\n NO GPU!")
    print("Go to: Runtime → Change runtime type → T4 GPU")


PyTorch: 2.9.0+cu126
CUDA Available: True
GPU: Tesla T4
Memory: 15.8 GB

✅ GPU READY!


In [ ]:
!pip install -q ultralytics wandb onnx onnxruntime onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00


### Dataset Preparation

To begin, the YOLOv8 dataset, provided as a `yolo_dataset.zip` file, needs to be unzipped. This action extracts all necessary image and label files into a structured directory.

Once unzipped, the integrity and structure of the dataset are crucial for successful model training. The first step in verification is to locate the `data.yaml` file, which contains essential metadata about the dataset, including class names and paths to the training, validation, and testing sets.

Following this, the number of images in each of the `train`, `val`, and `test` directories is counted to ensure a complete and balanced dataset split. This verification step is vital to confirm that the dataset is properly organized and ready for the YOLOv8 model training process.

In [ ]:
!unzip /content/yolo_dataset.zip

Archive:  /content/yolo_dataset.zip
   creating: yolo_dataset/
   creating: yolo_dataset/test/
   creating: yolo_dataset/train/
   creating: yolo_dataset/val/
  inflating: yolo_dataset/data.yaml  
   creating: yolo_dataset/train/images/
   creating: yolo_dataset/train/labels/
  inflating: yolo_dataset/train/labels/aug_12_raw-250110_dc_s001_b2_15.txt  
  inflating: yolo_dataset/train/labels/aug_13_raw-250110_dc_s001_b5_2.txt  
  inflating: yolo_dataset/train/labels/aug_4_raw-250110_dc_s001_b3_4.txt  
  inflating: yolo_dataset/train/labels/raw-250110_dc_s001_b2_3.txt  
  inflating: yolo_dataset/train/labels/aug_1_raw-250110_dc_s001_b2_1.txt  
  inflating: yolo_dataset/train/labels/aug_4_raw-250110_dc_s001_b5_3.txt  
  inflating: yolo_dataset/train/labels/aug_9_raw-250110_dc_s001_b3_4.txt  
  inflating: yolo_dataset/train/labels/aug_7_raw-250110_dc_s001_b4_2.txt  
  inflating: yolo_dataset/train/labels/raw-250110_dc_s001_b3_3.txt  
  inflating: yolo_dataset/train/labels/aug_10_raw-250110_

In [ ]:
from ultralytics import YOLO
import wandb

from pathlib import Path

data_yaml = Path('/content/yolo_dataset/data.yaml')

if not data_yaml.exists():
    print("❌ Dataset not found!")
    print("\n💡 Run Notebook 1 first to prepare dataset!")
    print("   Or make sure /content/yolo_dataset exists")
else:
    print("data.yaml found")

    train_imgs = len(list(Path('/content/yolo_dataset/train/images').glob('*')))
    val_imgs = len(list(Path('/content/yolo_dataset/val/images').glob('*')))
    test_imgs = len(list(Path('/content/yolo_dataset/test/images').glob('*')))

    print(f"\n📊 Dataset Split:")
    print(f"   Train: {train_imgs} images")
    print(f"   Val: {val_imgs} images")
    print(f"   Test: {test_imgs} images")

    print(f"\n📄 data.yaml content:")
    with open(data_yaml, 'r') as f:
        print(f.read())

    if train_imgs > 0 and val_imgs > 0:
        print("DATASET READY FOR TRAINING!")
    else:
        print("Dataset incomplete!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
data.yaml found

📊 Dataset Split:
   Train: 142 images
   Val: 41 images
   Test: 21 images

📄 data.yaml content:
path: /content/yolo_dataset
train: train/images
val: val/images
test: test/images
nc: 3
names:
- light_blue
- dark_blue
- others

DATASET READY FOR TRAINING!


### Weights & Biases Integration

Weights & Biases (WandB) will be an integral tool for experiment tracking, visualization, and logging throughout our YOLOv8 model training process. It provides a robust platform to manage machine learning experiments.

To connect to the WandB platform, we will use the `wandb.login()` function. This authenticates our session, allowing us to interact with the WandB service.

The benefits of using WandB are numerous:
*   **Tracking Metrics**: Automatically log training and validation metrics (e.g., loss, precision, recall, mAP) over epochs.
*   **Visualizing Predictions**: Log sample predictions, allowing us to visually inspect how the model performs on validation images at different stages of training.
*   **Managing Runs**: Easily compare different training runs, including their configurations, hyperparameters, and results.
*   **Configuration Management**: Store all hyperparameters and configurations used for a specific run, ensuring reproducibility.
*   **Artifacts**: Save model checkpoints and other relevant files directly to WandB, creating a centralized repository for our project.

In [ ]:
wandb.login()

print("WandB login successful!")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: andreanagari (andreanagari-unemployed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB login successful!


In [ ]:
from ultralytics import YOLO
import wandb
from pathlib import Path
import cv2
import numpy as np

print("="*70)
print("🔥 STARTING TRAINING (ENHANCED WANDB LOGGING)")
print("="*70)

try:
    wandb.finish()
except:
    pass

run = wandb.init(
    project="bottleCap-detection",
    name="yolov8n-final2",
    config={
        "epochs": 150,
        "batch_size": 8,
        "image_size": 320,
        "architecture": "YOLOv8n",
        "strategy": "High Geometry Augmentation",
        "optimizer": "AdamW",
        "learning_rate": 0.001,
        "degrees": 90,
        "translate": 0.1,
        "scale": 0.6,
        "shear": 10,
        "flipud": 0.5,
        "fliplr": 0.5,
        "mosaic": 1.0,
        "mixup": 0.1,
    }
)

print(f"\n📊 Dashboard Link: {run.url}")

model = YOLO("yolov8n.pt")

def comprehensive_wandb_callback(trainer):

    if not wandb.run:
        return

    epoch = trainer.epoch

    train_losses = {
        "train/box_loss": trainer.loss_items[0].item() if len(trainer.loss_items) > 0 else 0,
        "train/cls_loss": trainer.loss_items[1].item() if len(trainer.loss_items) > 1 else 0,
        "train/dfl_loss": trainer.loss_items[2].item() if len(trainer.loss_items) > 2 else 0,
        "train/total_loss": trainer.tloss.mean().item() if trainer.tloss is not None else 0,
    }

    val_metrics = {}
    if hasattr(trainer, "metrics") and trainer.metrics:
        val_metrics = {
            "val/precision": trainer.metrics.get("metrics/precision(B)", 0),
            "val/recall": trainer.metrics.get("metrics/recall(B)", 0),
            "val/mAP50": trainer.metrics.get("metrics/mAP50(B)", 0),
            "val/mAP50-95": trainer.metrics.get("metrics/mAP50-95(B)", 0),
        }

    lr_dict = {"train/learning_rate": trainer.optimizer.param_groups[0]["lr"]}

    wandb.log({**train_losses, **val_metrics, **lr_dict, "epoch": epoch})

    if epoch % 10 == 0:
        try:
            log_predictions(trainer, epoch)
        except Exception as e:
            print("⚠️ Could not log predictions:", e)


def log_predictions(trainer, epoch):

    val_path = Path("/content/yolo_dataset/val/images")
    val_images = list(val_path.glob("*.jpg"))[:3]

    try:
        weight_path = trainer.best if hasattr(trainer, "best") else trainer.last
        infer_model = YOLO(weight_path)
    except:
        infer_model = YOLO(trainer.model.model)
        infer_model.eval()

    prediction_images = []

    for img_path in val_images:

        results = infer_model(str(img_path))[0]

        plotted = results.plot()
        plotted_rgb = cv2.cvtColor(plotted, cv2.COLOR_BGR2RGB)

        boxes_data = []
        for box in results.boxes:
            boxes_data.append({
                "position": {
                    "minX": float(box.xyxy[0][0]),
                    "minY": float(box.xyxy[0][1]),
                    "maxX": float(box.xyxy[0][2]),
                    "maxY": float(box.xyxy[0][3]),
                },
                "class_id": int(box.cls[0]),
                "box_caption": f"{infer_model.names[int(box.cls[0])]} {float(box.conf[0]):.2f}",
                "scores": {"confidence": float(box.conf[0])},
            })

        wandb_img = wandb.Image(
            plotted_rgb,
            caption=f"Epoch {epoch} - {img_path.name}",
            boxes={"predictions": {"box_data": boxes_data, "class_labels": infer_model.names}},
        )

        prediction_images.append(wandb_img)

    wandb.log({f"predictions/epoch_{epoch}": prediction_images})



model.add_callback("on_fit_epoch_end", comprehensive_wandb_callback)


print("\n🔥 Starting training...")
results = model.train(
    data="/content/yolo_dataset/data.yaml",
    imgsz=320,
    epochs=150,
    batch=8,
    patience=0,
    close_mosaic=10,
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    device=0,
    workers=2,
    val=True,
    save=True,
    save_period=10,
    verbose=True,
    seed=42,
    plots=True,
)


print("\n📦 Logging final artifacts...")

best_model_path = Path("/content/runs/detect/train2/weights/best.pt")
if best_model_path.exists():
    wandb.save(str(best_model_path))
    print("✅ Best model saved")

print(f"\n🔗 Dashboard: {run.url}")
wandb.finish()

🔥 STARTING TRAINING (ENHANCED WANDB LOGGING)



📊 Dashboard Link: https://wandb.ai/andreanagari-unemployed/bottleCap-detection/runs/khtuz98m

🔥 Starting training...
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/box_loss,▇▆▄██▇▄▅▅▄▅▄▄▃▃▆▅▃▃▄▃▃▄▃▃▂▂▄▂▄▂▃▂▂▂▂▂▃▁▁
train/cls_loss,█▆▅▅▄▄▃▃▄▄▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▃▃▂▂▂▂▂▂▂▁▂▁▂▁
train/dfl_loss,▇▆▅▄▆▃█▄█▇▆▃▃▅▅▄▃▇▄▃▄▅▄▅▅▅▄▁▃▃▃▁▃▄▃▄▂▃▅▃
train/learning_rate,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP50,▁███████████████████████████████████████
val/mAP50-95,▁▃▆▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
val/precision,▁▆▇█████████████████████████████████████
val/recall,▁▃▄▄▆▆███▇▇██▇███████████▇██████████████
epoch,149


In [ ]:
import onnx
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType
import time
import os
import numpy as np
from pathlib import Path

possible_paths = list(Path('/content/runs/detect').glob('*/weights/best.pt'))
if not possible_paths:
    raise FileNotFoundError("❌ Model best.pt tidak ditemukan! Pastikan training sudah selesai.")

best_model_path = max(possible_paths, key=os.path.getmtime)
print(f"✅ Menggunakan model: {best_model_path}")

from ultralytics import YOLO
model = YOLO(best_model_path)

print("\n🔄 Sedang Export ke ONNX...")
model.export(format='onnx', imgsz=320, dynamic=False, simplify=True)

onnx_path = str(best_model_path).replace('.pt', '.onnx')
quantized_path = str(best_model_path).replace('.pt', '_int8.onnx')

print("\n📉 Sedang Melakukan Kuantisasi (Int8)...")
quantize_dynamic(
    model_input=onnx_path,
    model_output=quantized_path,
    weight_type=QuantType.QUInt8
)

print(f"🎉 Selesai! Model tersimpan di:")
print(f"   Original  : {onnx_path} ({os.path.getsize(onnx_path)/1024/1024:.2f} MB)")
print(f"   Quantized : {quantized_path} ({os.path.getsize(quantized_path)/1024/1024:.2f} MB)")


print("\n" + "="*50)
print("🍓 SIMULASI PERFORMA RASPBERRY PI 5")
print("="*50)

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 4
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

session = ort.InferenceSession(quantized_path, sess_options, providers=['CPUExecutionProvider'])

input_name = session.get_inputs()[0].name
dummy_input = np.random.randn(1, 3, 320, 320).astype(np.float32)

print("🔥 Warming up engine...")
for _ in range(10):
    session.run(None, {input_name: dummy_input})

num_runs = 100
latencies = []

print(f"🏃 Running benchmark ({num_runs} iterasi)...")
for i in range(num_runs):
    start_time = time.time()
    session.run(None, {input_name: dummy_input})
    end_time = time.time()
    latencies.append((end_time - start_time) * 1000)

avg_latency = np.mean(latencies)
fps = 1000 / avg_latency
p95_latency = np.percentile(latencies, 95)

print("\n📊 HASIL PENGUJIAN (Estimasi):")
print(f"   Model       : YOLOv8 (Int8 Quantized)")
print(f"   Image Size  : 320x320")
print(f"   Rata-rata Latency : {avg_latency:.2f} ms")
print(f"   95% Latency       : {p95_latency:.2f} ms")
print(f"   Estimasi FPS      : {fps:.2f} FPS")

print("\nℹ️ Catatan:")
print("   - Angka ini adalah simulasi 'Upper Bound' (batas atas).")
print("   - Di Raspberry Pi 5 asli, performa mungkin sedikit berbeda tergantung thermal throttling.")
print("   - Namun, model Int8 ini jauh lebih ringan daripada model PyTorch asli.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.5 MB/s eta 0:00:00
✅ Menggunakan model: /content/runs/detect/train/weights/best.pt

🔄 Sedang Export ke ONNX...
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 7, 2100) (5.9 MB)

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.75...
ONNX: export success ✅ 0.8s, saved as '/content/runs/detect/train/weights/best.onnx' (11.6 MB)

Export complete (0.9s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.onnx imgsz=320  
Validate:        yolo val task=detect model=/content/runs/detect/tr


📉 Sedang Melakukan Kuantisasi (Int8)...
🎉 Selesai! Model tersimpan di:
   Original  : /content/runs/detect/train/weights/best.onnx (11.58 MB)
   Quantized : /content/runs/detect/train/weights/best_int8.onnx (3.08 MB)

🍓 SIMULASI PERFORMA RASPBERRY PI 5
🔥 Warming up engine...
🏃 Running benchmark (100 iterasi)...

📊 HASIL PENGUJIAN (Estimasi):
   Model       : YOLOv8 (Int8 Quantized)
   Image Size  : 320x320
   Rata-rata Latency : 107.80 ms
   95% Latency       : 183.01 ms
   Estimasi FPS      : 9.28 FPS

ℹ️ Catatan:
   - Angka ini adalah simulasi 'Upper Bound' (batas atas).
   - Di Raspberry Pi 5 asli, performa mungkin sedikit berbeda tergantung thermal throttling.
   - Namun, model Int8 ini jauh lebih ringan daripada model PyTorch asli.


In [ ]:
from ultralytics import YOLO
import wandb
from pathlib import Path
import cv2
import numpy as np

print("="*70)
print("🔥 STARTING TRAINING (ENHANCED WANDB LOGGING)")
print("="*70)

try:
    wandb.finish()
except:
    pass

run = wandb.init(
    project="bottleCap-detection",
    name="yolov8n-128",
    config={
        "epochs": 150,
        "batch_size": 16,
        "image_size": 128,
        "architecture": "YOLOv8n",
        "strategy": "High Geometry Augmentation",
        "optimizer": "AdamW",
        "learning_rate": 0.001,
        "degrees": 90,
        "translate": 0.1,
        "scale": 0.6,
        "shear": 10,
        "flipud": 0.5,
        "fliplr": 0.5,
        "mosaic": 1.0,
        "mixup": 0.1,
    }
)

print(f"\n📊 Dashboard Link: {run.url}")

model = YOLO("yolov8n.pt")


def comprehensive_wandb_callback(trainer):

    if not wandb.run:
        return

    epoch = trainer.epoch

    train_losses = {
        "train/box_loss": trainer.loss_items[0].item() if len(trainer.loss_items) > 0 else 0,
        "train/cls_loss": trainer.loss_items[1].item() if len(trainer.loss_items) > 1 else 0,
        "train/dfl_loss": trainer.loss_items[2].item() if len(trainer.loss_items) > 2 else 0,
        "train/total_loss": trainer.tloss.mean().item() if trainer.tloss is not None else 0,
    }

    val_metrics = {}
    if hasattr(trainer, "metrics") and trainer.metrics:
        val_metrics = {
            "val/precision": trainer.metrics.get("metrics/precision(B)", 0),
            "val/recall": trainer.metrics.get("metrics/recall(B)", 0),
            "val/mAP50": trainer.metrics.get("metrics/mAP50(B)", 0),
            "val/mAP50-95": trainer.metrics.get("metrics/mAP50-95(B)", 0),
        }

    lr_dict = {"train/learning_rate": trainer.optimizer.param_groups[0]["lr"]}

    wandb.log({**train_losses, **val_metrics, **lr_dict, "epoch": epoch})


    if epoch % 10 == 0:
        try:
            log_predictions(trainer, epoch)
        except Exception as e:
            print("⚠️ Could not log predictions:", e)


def log_predictions(trainer, epoch):

    val_path = Path("/content/yolo_dataset/val/images")
    val_images = list(val_path.glob("*.jpg"))[:3]


    try:
        weight_path = trainer.best if hasattr(trainer, "best") else trainer.last
        infer_model = YOLO(weight_path)
    except:
        infer_model = YOLO(trainer.model.model)
        infer_model.eval()

    prediction_images = []

    for img_path in val_images:

        results = infer_model(str(img_path))[0]

        plotted = results.plot()
        plotted_rgb = cv2.cvtColor(plotted, cv2.COLOR_BGR2RGB)

        boxes_data = []
        for box in results.boxes:
            boxes_data.append({
                "position": {
                    "minX": float(box.xyxy[0][0]),
                    "minY": float(box.xyxy[0][1]),
                    "maxX": float(box.xyxy[0][2]),
                    "maxY": float(box.xyxy[0][3]),
                },
                "class_id": int(box.cls[0]),
                "box_caption": f"{infer_model.names[int(box.cls[0])]} {float(box.conf[0]):.2f}",
                "scores": {"confidence": float(box.conf[0])},
            })

        wandb_img = wandb.Image(
            plotted_rgb,
            caption=f"Epoch {epoch} - {img_path.name}",
            boxes={"predictions": {"box_data": boxes_data, "class_labels": infer_model.names}},
        )

        prediction_images.append(wandb_img)

    wandb.log({f"predictions/epoch_{epoch}": prediction_images})



model.add_callback("on_fit_epoch_end", comprehensive_wandb_callback)


print("\n🔥 Starting training...")
results = model.train(
    data="/content/yolo_dataset/data.yaml",
    imgsz=128,
    epochs=150,
    batch=16,
    patience=10,
    freeze=0,
    close_mosaic=10,
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    device=0,
    workers=2,
    val=True,
    save=True,
    save_period=10,
    verbose=True,
    seed=42,
    plots=True,
)

print("\n📦 Logging final artifacts...")

best_model_path = Path("/content/runs/detect/train2/weights/best.pt")
if best_model_path.exists():
    wandb.save(str(best_model_path))
    print("✅ Best model saved")

print(f"\n🔗 Dashboard: {run.url}")
wandb.finish()

🔥 STARTING TRAINING (ENHANCED WANDB LOGGING)



📊 Dashboard Link: https://wandb.ai/andreanagari-unemployed/bottleCap-detection/runs/vurlkwu3

🔥 Starting training...
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=F

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")



📦 Logging final artifacts...
✅ Best model saved

🔗 Dashboard: https://wandb.ai/andreanagari-unemployed/bottleCap-detection/runs/vurlkwu3


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
train/box_loss,▆█▆▄▄▅▃▄▃▃▃▃▅▃█▃▄▄▃▅▂▃▃▂▂▂▂▁▃▃▂▁▂▂▁▂▂▃▁▃
train/cls_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
train/dfl_loss,█▅▆▅▄▃▅▂▄▂▄▃▃▃▄▁▂▄▃▄▄▂▁▄▄▃▄▄▃▄▂▃▂▃▂▂▃▄▃▄
train/learning_rate,█▇▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_loss,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP50,▁▄▄▅▆▆▆▆▆▇▇▇▇███████████████████████████
val/mAP50-95,▁▁▁▁▂▅▅▅▅▆▆▆▇▇▇▇▇▆▇███▇▇▇▇▇▇█▇▇▇▇███████
val/precision,▁▁▁▄▅▆▅▅▆▆▆▆▇▇▇████▇████████████████████
val/recall,▁▃▇▇▇▇██████▇▇▇█████████▇███████████████
epoch,70


In [ ]:
import onnx
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType
import time
import os
import numpy as np
from pathlib import Path
from ultralytics import YOLO

model_path = Path("/content/runs/detect/train2/weights/best.pt")

if not model_path.exists():
    print(f"⚠️ Model tidak ditemukan di: {model_path}")
    print("   Mencari alternatif di folder runs...")
    possible_paths = list(Path('/content/runs/detect').glob('*/weights/best.pt'))
    if possible_paths:
        model_path = max(possible_paths, key=os.path.getmtime)
        print(f"✅ Ditemukan model alternatif: {model_path}")
    else:
        raise FileNotFoundError("❌ Tidak ada model best.pt ditemukan! Pastikan training 128x128 sudah selesai.")
else:
    print(f"✅ Menggunakan model: {model_path}")

model = YOLO(model_path)
target_size = 128

print(f"\n📦 Exporting ke ONNX ({target_size}x{target_size})...")
onnx_path = f"/content/yolo_tiny_{target_size}.onnx"

model.export(
    format='onnx',
    imgsz=target_size,
    dynamic=False,
    simplify=True
)

exported_onnx = model_path.with_suffix('.onnx')
if exported_onnx.exists():
    os.rename(exported_onnx, onnx_path)

print("\n📉 Melakukan Kuantisasi Int8...")
quantized_path = f"/content/yolo_tiny_{target_size}_int8.onnx"

quantize_dynamic(
    model_input=onnx_path,
    model_output=quantized_path,
    weight_type=QuantType.QUInt8
)

original_size = os.path.getsize(onnx_path) / (1024*1024)
quantized_size = os.path.getsize(quantized_path) / (1024*1024)

print(f"✅ Kompresi Selesai!")
print(f"   Original  : {original_size:.2f} MB")
print(f"   Quantized : {quantized_size:.2f} MB (Lebih kecil {original_size/quantized_size:.1f}x)")

print("\n" + "="*50)
print(f"⚡ BENCHMARK SPEED TEST (Simulasi RPi 5)")
print("="*50)

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 4
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

session = ort.InferenceSession(quantized_path, sess_options, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name

dummy_input = np.random.randn(1, 3, target_size, target_size).astype(np.float32)

print("🔥 Warming up...")
for _ in range(20):
    session.run(None, {input_name: dummy_input})

num_runs = 200
print(f"🏃 Running {num_runs} inferences...")
latencies = []

for i in range(num_runs):
    start = time.time()
    session.run(None, {input_name: dummy_input})
    end = time.time()
    latencies.append((end - start) * 1000)

avg_latency = np.mean(latencies)
p95_latency = np.percentile(latencies, 95)
fps = 1000 / avg_latency

print(f"\n📊 HASIL AKHIR ({target_size}x{target_size} Int8):")
print(f"   Rata-rata Latency : {avg_latency:.2f} ms")
print(f"   95% Latency       : {p95_latency:.2f} ms")
print(f"   Estimasi FPS      : {fps:.2f} FPS")

✅ Menggunakan model: /content/runs/detect/train2/weights/best.pt

📦 Exporting ke ONNX (128x128)...
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train2/weights/best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 7, 336) (5.9 MB)

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.75...
ONNX: export success ✅ 1.4s, saved as '/content/runs/detect/train2/weights/best.onnx' (11.5 MB)

Export complete (1.7s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train2/weights/best.onnx imgsz=128  
Validate:        yolo val task=detect model=/content/runs/detect/train2/weights/best.onnx imgsz=128 data=/content/yolo_dataset/data.yaml  
Visualize:       https://netron.app



📉 Melakukan Kuantisasi Int8...
✅ Kompresi Selesai!
   Original  : 11.54 MB
   Quantized : 3.04 MB (Lebih kecil 3.8x)

⚡ BENCHMARK SPEED TEST (Simulasi RPi 5)
🔥 Warming up...
🏃 Running 200 inferences...

📊 HASIL AKHIR (128x128 Int8):
   Rata-rata Latency : 28.73 ms
   95% Latency       : 50.85 ms
   Estimasi FPS      : 34.81 FPS

💡 Kesimpulan:
⚠️ Masih agak berat. Pastikan nanti di RPi menggunakan implementasi C++ (NCNN) bukan Python.
